In [2]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [4]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Read Parquet and Extract Schema") \
    .getOrCreate()

# Path to your Parquet file
parquet_file_path = "file:///home/talentum/datapar/*.parquet"

# Read the Parquet file into a DataFrame
df = spark.read.parquet(parquet_file_path)

# Extract and print the schema
schema = df.schema
print("Schema of the Parquet file:")
print(schema)


Schema of the Parquet file:
StructType(List(StructField(TransactionID,StringType,true),StructField(CustomerID,StringType,true),StructField(CustGender,StringType,true),StructField(CustAccountBalance,FloatType,true),StructField(TransactionDate,DateType,true),StructField(TransactionAmountINR,FloatType,true),StructField(CustomerAge,IntegerType,true),StructField(CustLocation,StringType,true)))


In [5]:
from pyspark.sql import SparkSession

# Initialize Spark Session with Hive support
spark = SparkSession.builder \
    .appName("Spark Hive Integration") \
    .config("spark.sql.catalogImplementation", "hive") \
    .enableHiveSupport() \
    .getOrCreate()

# Describe the Hive table
spark.sql("DESCRIBE project.bank_customer_segmentation").show()

# Read a Parquet file into a DataFrame
parquet_file_path = "file:///home/talentum/datapar/*.parquet"
df = spark.read.parquet(parquet_file_path)

# Write Data to the Hive table
df.write.mode("overwrite").saveAsTable("project.bank_customer_segmentation")

# Read the data from the Hive table
df_from_hive = spark.sql("SELECT * FROM project.bank_customer_segmentation")

# Show the top 50 rows
df_from_hive.show(5)

# Stop the Spark session
#spark.stop()


+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|      transaction_id|   string|   null|
|         customer_id|   string|   null|
|         cust_gender|   string|   null|
|cust_account_balance|   double|   null|
|    transaction_date|     date|   null|
|transaction_amoun...|   double|   null|
|        customer_age|      int|   null|
|       cust_location|   string|   null|
+--------------------+---------+-------+

+-------------+----------+----------+------------------+---------------+--------------------+-----------+------------+
|TransactionID|CustomerID|CustGender|CustAccountBalance|TransactionDate|TransactionAmountINR|CustomerAge|CustLocation|
+-------------+----------+----------+------------------+---------------+--------------------+-----------+------------+
|           T1|  C5841053|         F|          17819.05|     2016-08-02|                25.0|         22|  JAMSHEDPUR|
|           T3|  C4417068|     